In [ ]:
from scipy.optimize import curve_fit, OptimizeWarning
import matplotlib.pyplot as plt
from functools import partial
import multiprocessing
from tqdm import tqdm
import numpy as np
import warnings
import zipfile
import imageio
import pickle
import tqdm
import os
import io
from params import p
project_name = "sample"

In [ ]:
# Set the project name to before loading the parameters
p.proj_name = project_name
# When we call p.load(), it should attempt to load "params_sample.toml"
p.load()
# Print the parameters to verify they are loaded correctly
print(p)

In [ ]:
# core

In [ ]:
def makeMovie(IMG1, prD, psinum, fps):
    dim = int(np.sqrt(max(IMG1.shape)))  # window size
    nframes = IMG1.shape[1]
    images = -IMG1
    gif_path = os.path.join(p.out_dir, "topos", f"PrD_{prD + 1}", f'psi_{psinum + 1}.gif')
    zip_path = os.path.join(p.out_dir, "topos", f"PrD_{prD + 1}", f'psi_{psinum + 1}.zip')
    frame_dt = 1.0/fps
    with zipfile.ZipFile(zip_path, 'w') as fzip:
        with imageio.get_writer(gif_path, mode='I', duration=frame_dt) as writer:
            for i in range(nframes):
                img = images[:, i].reshape(dim, dim)
                frame = np.round(255 * (img - np.min(img)) / (np.max(img) - np.min(img))).astype(np.uint8)
                frame_path = 'frame{:02d}.png'.format(i)
                b = io.BytesIO()
                imageio.imwrite(b, frame, format='png')
                b.seek(0)
                fzip.writestr(frame_path, b.read())
                writer.append_data(frame)

In [ ]:
class NullEmitter:
    """
    A class that provides a no-operation (no-op) implementation of an emitter.

    This class is designed to be used in contexts where an emitter is required by the interface,
    but no actual emitting action is desired. It effectively serves as a placeholder or a stub
    that satisfies the requirement of having an emitter without performing any operation.
    """
    
    def emit(self, percent):
        """
        A no-operation implementation of the emit method.

        This method is intended to fulfill the interface requirement for an emitting action
        without performing any actual work. It can be used to ignore progress updates or
        other emitting actions in a safe and controlled manner.

        Parameters:
        - percent (any): This parameter is accepted to match the expected interface of an
          emitter method, but it is not used within the method. Any value passed to this
          parameter will be ignored.

        Returns:
        None
        """
        pass

In [ ]:
def debug_print(msg: str=""):
    """
    Prints a debug message along with the caller's stack trace.

    Parameters:
    - msg (str): The debug message to print. If empty, only the stack trace is printed.

    Returns:
    None
    """
    if msg:
        print(msg)
    stack = traceback.format_stack()
    print(stack[-2].split('\n')[0])

In [ ]:
def fin1(filename):
    """
    Reads and returns the data from a pickle file.

    This function attempts to open a file in binary read mode and deserialize its contents using
    pickle. If an exception occurs during this process, it logs the exception message using
    `debug_print` and returns None.

    Parameters:
    - filename (str): The path to the file to be read.

    Returns:
    - The deserialized data from the file if successful, None otherwise.
    """
    with open(filename, 'rb') as f:
        try:
            data = pickle.load(f)
            return data
        except Exception as e:
            debug_print(str(e))
            return None

In [ ]:
def makeMovie(IMG1, prD, psinum, fps):
    """
    Generates a movie (GIF format) from a series of images represented by 'IMG1' and saves it along
    with the individual frames as a ZIP archive. The movie and ZIP file are stored in specific directories
    based on the provided parameters.

    Parameters:
        IMG1 (np.ndarray): A 2D numpy array where each column represents a flattened image to be included
                           in the movie. The number of columns determines the number of frames.
        prD (int): An index representing a specific dimensionality reduction or processing step. Used to
                   organize the output files into directories.
        psinum (int): The index of the current eigenfunction being processed, used for naming the output files.
        fps (int): Frames per second, determining the playback speed of the generated movie.

    Note:
        - The function computes the dimension ('dim') of the square images based on the shape of 'IMG1'.
        - It normalizes the image data to the range [0, 255] for proper visualization.
        - The output files are named using 'prD' and 'psinum' to reflect the processing step and eigenfunction index.
        - The movie is saved as a GIF file, and the individual frames are archived in a ZIP file,
          both stored under the 'topos' directory within the output directory specified by 'p.out_dir'.

    Raises:
        IOError: If there is an issue with file writing operations or directory creation.
        ValueError: If 'IMG1' does not contain valid image data or if the dimensions are not appropriate for
                    reshaping into square images.

    Example usage:
        makeMovie(IMG1_array, 0, 1, 24) # Generates a movie from 'IMG1_array', for prD=0, psinum=1, at 24 fps.
    """
    dim = int(np.sqrt(max(IMG1.shape)))  # window size
    nframes = IMG1.shape[1]
    images = -IMG1
    gif_path = os.path.join(p.out_dir, "topos", f"PrD_{prD + 1}", f'psi_{psinum + 1}.gif')
    zip_path = os.path.join(p.out_dir, "topos", f"PrD_{prD + 1}", f'psi_{psinum + 1}.zip')
    frame_dt = 1.0/fps
    with zipfile.ZipFile(zip_path, 'w') as fzip:
        with imageio.get_writer(gif_path, mode='I', duration=frame_dt) as writer:
            for i in range(nframes):
                img = images[:, i].reshape(dim, dim)
                frame = np.round(255 * (img - np.min(img)) / (np.max(img) - np.min(img))).astype(np.uint8)
                frame_path = 'frame{:02d}.png'.format(i)
                b = io.BytesIO()
                imageio.imwrite(b, frame, format='png')
                b.seek(0)
                fzip.writestr(frame_path, b.read())
                writer.append_data(frame)

In [ ]:
def _construct_input_data(N):
    """
    Constructs a list of input data parameters for processing, based on the number of
    dimensionality reduction steps or projections specified by 'N'. Each element in the
    list represents a set of parameters for a single projection, specifically the projection
    index 'prD'.

    This function checks the existence of an image file associated with each projection
    to determine if the projection has already been processed. If the file exists, the
    projection is skipped; otherwise, it's included in the list for processing.

    Parameters:
        N (int): The total number of projections or dimensionality reduction steps to be considered.

    Returns:
        list: A list where each element is a list containing a single element, the projection index 'prD',
              for projections that have not yet been processed.

    Note:
        - The function generates paths to check for the existence of 'class_avg.png' files within directories
          named according to each projection index 'prD'. These files are expected to exist in a 'topos'
          subdirectory within the output directory specified by 'p.out_dir'.
        - Only projections for which the corresponding 'class_avg.png' file does not exist are included in
          the return list, indicating that these projections require processing.

    Example usage:
        input_data = _construct_input_data(5) # Prepares input data for up to 5 projections, excluding already processed.
    """
    ll = []
    for prD in range(N):
        image_file = '{}/topos/PrD_{}/class_avg.png'.format(p.out_dir, prD + 1)
        if os.path.exists(image_file):
            continue
        ll.append([prD])
    return ll

In [ ]:
def movie(input_data, out_dir, dist_file, psi2_file, fps):
    """
    Generates movies and topological images from NLSA analysis results for each eigenfunction and saves
    a class average image. The function processes a specified projection index 'prD', retrieves data for
    each eigenfunction, generates a movie, and saves topological images and a class average image.

    Parameters:
        input_data (list): A list containing the projection index 'prD' as its first element.
        out_dir (str): The output directory where the generated files will be saved.
        dist_file (str): Path to the distribution file, used here for documentation consistency but not directly.
        psi2_file (str): The base path for psi files, to which 'prD' and 'psi' identifiers will be appended.
        fps (int): Frames per second for the generated movies.

    Note:
        - The function iterates through each eigenfunction specified by 'p.num_psis', retrieves the corresponding
          data, generates a movie for each, and saves topological images.
        - The class average image is saved based on data fetched from a distribution file specific to the
          current projection index 'prD'.
        - It is important that 'p.get_dist_file(prD)' and 'fin1' are defined and accessible within the scope
          of this function, as they are used to retrieve the necessary data.
        - The function utilizes 'matplotlib' for generating and saving topological images, which requires
          proper handling of figures and axes to ensure that resources are appropriately managed.

    Raises:
        IOError: If there is an issue accessing the input files or writing the output files.
        ValueError: If the input data or the retrieved data is invalid or cannot be processed as expected.

    Example usage:
        movie([0], '/path/to/out_dir', '/path/to/dist_file', '/path/to/psi2_file', 24)
        # This will process the projection with index 0, generating movies and images at the specified location.
    """
    prD = input_data[0]
    dist_file1 = p.get_dist_file(prD)
    # Fetching NLSA outputs and making movies
    IMG1All = []
    Topo_mean = []
    for psinum in range(p.num_psis):
        psi_file1 = psi2_file + 'prD_{}'.format(prD) + '_psi_{}'.format(psinum)
        data = fin1(psi_file1)
        IMG1All.append(data['IMG1'])
        Topo_mean.append(data['Topo_mean'])
        # make movie
        makeMovie(IMG1All[psinum], prD, psinum, fps)
        # write topos
        topo = Topo_mean[psinum]
        dim = int(np.sqrt(topo.shape[0]))

        fig2 = plt.figure(frameon=False)
        ax2 = fig2.add_axes([0, 0, 1, 1])
        ax2.axis('off')
        ax2.set_title('')
        ax2.get_xaxis().set_visible(False)
        ax2.get_yaxis().set_visible(False)
        ax2.imshow(topo[:, 1].reshape(dim, dim), cmap=plt.get_cmap('gray'))
        image_file = f'{out_dir}/topos/PrD_{prD + 1}/topos_{psinum + 1}.png'
        fig2.savefig(image_file, bbox_inches='tight', dpi=100, pad_inches=-0.1)
        ax2.clear()
        fig2.clf()
        plt.close(fig2)
    # write class avg image
    data = fin1(dist_file1)
    avg = data['imgAvg']
    fig3 = plt.figure(frameon=False)
    ax3 = fig3.add_axes([0, 0, 1, 1])
    ax3.axis('off')
    ax3.set_title('')
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    ax3.imshow(avg, cmap=plt.get_cmap('gray'))
    image_file = f'{out_dir}/topos/PrD_{prD + 1}/class_avg.png'
    fig3.savefig(image_file, bbox_inches='tight', dpi=100, pad_inches=-0.1)
    ax3.clear()
    fig3.clf()
    plt.close(fig3)

In [ ]:
def op(*argv):
    """
    Executes the process of generating 2D movies from NLSA analysis results across multiple projection
    indices. This function manages the entire workflow, including loading configurations, setting up
    multiprocessing, and iterating over a set of input data to generate movies for each specified projection.

    Parameters:
        *argv: Variable length argument list. The presence of any argument enables GUI progress reporting.
               The first argument, if present, is expected to be an instance capable of emitting progress
               updates to a GUI.

    Note:
        - Initializes multiprocessing with 'fork' start method to ensure compatibility across different platforms.
        - Determines whether to use a GUI for progress updates based on the presence of arguments in 'argv'.
        - Constructs input data based on the number of jobs specified in the global parameter object 'p'.
        - Uses a partial function 'movie_local' tailored to each job's specific parameters for generating movies.
        - Handles both single-core and multi-core execution environments, defaulting to sequential processing
          when only one CPU core is available, and utilizing a pool of worker processes otherwise.
        - Progress updates are emitted through 'progress4', which is determined based on 'argv' or defaults
          to a 'NullEmitter' in non-GUI environments.

    Raises:
        Exception: If there are issues with multiprocessing initialization, data processing, or file operations.

    Example usage:
        op() # Non-GUI mode, suitable for command-line execution.
        op(progress_emitter) # GUI mode, with 'progress_emitter' for progress updates.
    """
    print("Making the 2D movies...")
    p.load()
    multiprocessing.set_start_method('fork', force=True)
    use_gui_progress = len(argv) > 0

    input_data = _construct_input_data(p.numberofJobs)
    n_jobs = len(input_data)
    progress4 = argv[0] if use_gui_progress else NullEmitter()
    movie_local = partial(movie, out_dir=p.out_dir, dist_file=p.dist_file, psi2_file=p.psi2_file, fps=p.fps)
    if p.ncpu == 1:  # avoids the multiprocessing package
        for i, datai in tqdm.tqdm(enumerate(input_data), total=n_jobs, disable=use_gui_progress):
            movie_local(datai)
            progress4.emit(int(99 * i / n_jobs))
    else:
        with multiprocessing.Pool(processes=p.ncpu) as pool:
            for i, _ in tqdm.tqdm(enumerate(pool.imap_unordered(movie_local, input_data)),
                                  total=n_jobs, disable=use_gui_progress):
                progress4.emit(int(99 * i / n_jobs))
    p.save()
    progress4.emit(100)

op()

### Step 1: Load Configuration

In [ ]:
p.load()
print(p)

### Step 2: Construct Input Data

In [ ]:
input_data = _construct_input_data(p.numberofJobs)
print(input_data)

### Step 3: Define the Function

In [ ]:
movie_local = partial(movie, 
                      out_dir=p.out_dir, 
                      dist_file=p.dist_file, 
                      psi2_file=p.psi2_file, 
                      fps=p.fps)

### Step 4: Process Datasets Sequentially

In [ ]:
n_jobs = len(input_data)
for i, datai in tqdm.tqdm(enumerate(input_data), total=n_jobs):
    movie_local(datai)

### Step 5: Save Configuration

In [ ]:
p.resProj = 3
p.save()